# ocr

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
from utils import CTCLabelConverter

## data

In [2]:
from data.dataset import OCRDataset
from data.dataset import get_vocab

/home/pc/miniconda3/envs/mlops/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
image_path = "/home/pc/Desktop/coding/vision/lib_ocr/trainning_images"
VOCAB = get_vocab(image_path)
toy_dataset = OCRDataset(image_path)


In [4]:
convert = CTCLabelConverter(VOCAB, device)

## model

In [5]:
from models.backbones import ResNet_FeatureExtractor
from models.pred_modules import CTC_Predictor

In [6]:
from torch import nn
class OCR_model_T1(nn.Module):
    def __init__(self, input_channels, num_classes):
        super().__init__()
        self.FeatureExtraction = ResNet_FeatureExtractor(input_channels,512)
        self.AdaptiveAvgPool = nn.AdaptiveAvgPool2d((None, 1))
        self.Predictor = CTC_Predictor(512, num_classes)
        
        
    def forward(self, input):
        # import pdb; pdb.set_trace()
        # input: [b, c, h, w] 
        # input: torch.Size([8, 1, 32, 100])

        visual_feature = self.FeatureExtraction(input) 
        # visual_feature: torch.Size([8, 512, 1, 26])

        visual_feature = self.AdaptiveAvgPool(visual_feature.permute(0, 3, 1, 2))  # [b, c, h, w] -> [b, w, c, h]
        # visual_feature: torch.Size([8, 26, 512, 1])

        visual_feature = visual_feature.squeeze(3)
        # visual_feature: torch.Size([8, 26, 512])

        prediction = self.Predictor(visual_feature.contiguous())
        # prediction: torch.Size([8, 26, 94])
        return prediction
    
model = OCR_model_T1(1, len(convert.character)).to(device)


In [7]:
import torch
from data.dataset import AlignCollate
align_collate = AlignCollate()
batch_size = 32
data_loader = torch.utils.data.DataLoader(toy_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=True,
                                          pin_memory=True,
                                            num_workers=4, # delete this line if you debug
                                          collate_fn=align_collate)
# import pdb; pdb.set_trace()
sample = next(iter(data_loader))

In [8]:
sample[0].shape

torch.Size([32, 1, 32, 100])

In [9]:
sample[1]

['Mua',
 'Ford Mustang',
 'ngoài. Có',
 'TTGT',
 'ha cà',
 'nào?. Mua nhà',
 '500m³ cát trái',
 'bà',
 'bóng',
 'trinh',
 'trong tháng',
 'thu hút',
 'Long -',
 'Bí.',
 'xóa',
 'Venezuela',
 'cao',
 'giá vé',
 'thi',
 'Bình xin phá',
 'y',
 'Nhìn',
 'dày khi',
 'Mê Linh',
 'công',
 'ký',
 'Lào Cai.',
 'công an.',
 "xanh'",
 'sát',
 'pháp lý.',
 'Sài Gòn.']

In [10]:

# with torch.inference_mode():
#     y_pred = model(sample[0])
#     print(y_pred.shape)

## traning loop

In [11]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [12]:
from torch.optim import Adam
from torch.nn import CTCLoss
from tqdm.auto import tqdm

optimizer = Adam(model.parameters(), lr=1e-3)
ctc_loss = CTCLoss().to(device)  # Ensure blank index matches your vocab
model.train()

for epoch in tqdm(range(30)):
    for idx, (image, labels) in tqdm(enumerate(data_loader)):
        # Encode labels
        text, length = convert.encode(labels, batch_max_length=len(VOCAB))
        
        
        # Forward pass
        preds = model(image.to(device))
        
        # Ensure `preds_size` matches batch size
        batch_size = image.size(0)
        preds_size = torch.IntTensor([preds.size(1)] * batch_size)  # Each sequence has `preds.size(1)` timesteps
        
        preds = preds.log_softmax(2).permute(1, 0, 2)  
        
        # Compute CTC loss
        loss = ctc_loss(preds, text, preds_size, length)
        
        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)  # gradient clipping with 5 (Default)
        optimizer.step()
        
        # Logging
        if idx % 100 == 0:
            print(f"Epoch: {epoch}, Batch: {idx}, Loss: {loss.item()}")
    

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0, Batch: 0, Loss: 29.564260482788086


32it [00:04,  7.24it/s]
  3%|▎         | 1/30 [00:04<02:09,  4.46s/it]

Epoch: 1, Batch: 0, Loss: 4.744544982910156


32it [00:04,  7.92it/s]
  7%|▋         | 2/30 [00:08<01:59,  4.25s/it]

...

32it [00:04,  7.90it/s]
 93%|█████████▎| 28/30 [01:54<00:08,  4.10s/it]

Epoch: 28, Batch: 0, Loss: 1.1247371435165405


32it [00:04,  7.90it/s]
 97%|█████████▋| 29/30 [01:58<00:04,  4.10s/it]

Epoch: 29, Batch: 0, Loss: 0.9792147874832153


32it [00:04,  7.84it/s]
100%|██████████| 30/30 [02:02<00:00,  4.09s/it]
